# import

In [ ]:
import sys
sys.path.append("/workspace/NAS/home/projects/ml_module/hsi_auto/")

from hsi_auto.utils.file_io import load_yaml
from hsi_auto.preprocessing.dataset_split import load_all_datasets_parallel


from direct_train import train_directnet_with_datasetlist
from direct_predict import run_predict_direct
from evaluate_directnet import evaluate_dataset_direct

cfg = load_yaml("directnet_config.yaml")


# 저장된 라벨 호출

In [ ]:
# 원본 라벨 그대로
dataset_list = load_all_datasets_parallel(
    base_dir="/workspace/NAS/home/crew/jinho/vision/Base_dataset/250514-level4(피복관)-test",
    source="lumo",
    use_pseudo_label=False,          # ✅ 명시적으로 pseudo label 사용
    normal_classes= [1,2,3,4],
    anomaly_classes=[5],
    ignore_classes=[0],
    visualize=False,
    max_workers=16
)

print(f"✅ 전체 데이터셋 수: {len(dataset_list)}개")

# from hsi_auto.utils.plot import plot_label_overlay
# for idx, sample in enumerate(dataset_list):

#     print(f"\n🔍 샘플 {idx + 1} 시각화 중...")
#     plot_label_overlay(sample["corrected"], sample["label"], title=f"Sample {idx + 1}")


#     print(f"\n🔍 샘플 {idx + 1} 시각화 중...")
#     plot_label_overlay(sample["corrected"], sample["label"], title=f"Sample {idx + 1}")




# 훈련

In [ ]:
augment_cfg = {
    "noise_prob": 0.6,               # Gaussian 노이즈 (센서 잡음 모사)
    "noise_std": 0.015,              # 노이즈 강도

    "shift_prob": 0.2,               # 밴드 시프트 (파장 정렬 오류 모사)
    "max_shift": 5,                  # 이동 범위

    "dropout_prob": 0.3,             # 일부 밴드 결손 (센서 밴드 드롭 또는 고장 모사)
    "band_dropout_prob": 0.1,        # 개별 밴드 드롭 확률
}


# ===== 사용 예시 =====
cfg = load_yaml("directnet_config.yaml")

train_directnet_with_datasetlist(
    config=cfg,
    dataset_list=dataset_list,       # 사전 정규화된 dataset_list (H×W×C + new_label 포함)
    epoches=10,                      # 반복 epoch 수 (없으면 config에서 자동 결정)
    device="cuda:0",                # 사용 디바이스
    augment=False,                   # augmentation 여부
    augment_params=augment_cfg      # 위에서 정의한 dict
)

In [ ]:
import torch
torch.cuda.empty_cache()   # PyTorch GPU 캐시 해제

# 평가

In [ ]:
cfg["epochs"] = 10  # 로그 경로 생성용(경로 맞출 때 필수)
model_path = "./checkpoints/MyData/DirectNet_batch_size_128_epoch_10_learning_rate_0.0001_win_out_19_gpu_ids_0/model/DirectNet_MyData_best.pt"

oa, rpt = evaluate_dataset_direct(
    dataset_list=dataset_list,
    config=cfg,
    model_path=model_path,
    device="cuda:0",
    visualize=True,
    percentile=98,
    save=False
)

print("평균 AUC:", oa)
for i, report in enumerate(rpt["sample_reports"]):
    print(f"[{i}] {report['path']}")
    print(f"  AUC: {report['auc']:.4f}")
    print(f"  threshold: {report['threshold']:.5f}")
    print(f"  n_total: {report['n_total']}, n_anomaly: {report['n_anomaly']}, n_pred_anomaly: {report['n_pred_anomaly']}")
    print(f"  confusion_matrix:\n{report['cm']}\n")


# 추론

In [ ]:
cfg["epochs"] = 10  # 로그 디렉토리 경로 생성용으로 사용됨

results = run_predict_direct(
    dataset_list=dataset_list,
    config=cfg,
    device="cuda:0",
    ckpt_tag="10",            # 저장된 모델의 epoch 이름
    visualize=True,
    percentile=95             # 이상 판단 기준을 상위 1%로 설정
)


In [ ]:
import torch
torch.cuda.empty_cache()   # PyTorch GPU 캐시 해제